# 처음 배우는 셸 스크립트#7

## 3부 예제와 함께 활용하는 셸 스크립트 활용
시스템 운영, 개발환경 구축, 클라우드 시스템 운영 시 발생하는 다양한 상황에서 셸 스크립트를 활용한다.  
상황 -> 방법 찾기 -> 스크립트 생성 -> 문제 해결  

## 11 모니터링
대부분 모니터링 솔루션이나 시스템을 이용하여 모니터링을 하지만, 그런 상황이 되지 않을 경우에는 셸 스크립트로 모니터링 한다.

### 11.1 IPItool을 이용하여 서버 전원 체크할 때
##### 상황 
물리 서버의 전원이 정상인지 모니터링하는 일은 매우 중요하다. 대부분의 서버 업체는 전용 네트워크 인터페이스를 통해 서버의 전원을 제어할 수 있는 시스템을 제공한다.  
리눅스에서는 IPMItool을 이용하여 시스템의 전원 상태를 체크하고 전원을 제어할 수 있다.  

##### 방법 찾기
IPMI는 Intelligent Platform Management Interface의 약자로 하드웨어를 원격으로 관리하기 위한 인터페이스이다.  
IPMI를 사용하기 위해서는 서버에 구성된 IPMI용 포트에 네트워크가 연결되어 있어야 한다. 그리고 물리 서버 바이오스의 IPMI 정보에 해당 IP 및 포트, 그리고 사용자 계정과 패스워드가 설정되어 있어야 한다. 이런 구성 하에 관리 서버에서 클러스터링으로 묶인 서버들의 전원 상태를 확인할 수 있다.
필요한 정보  
- 모니터링하고자 하는 서버의 IPMI IP, Port, User ID, User Password
- 전원 체크를 위한 IPMI 명령어: ipmitool

프로세스
- 모니터링하고자 하는 서버의 IPMI IP 정보를 변수에 저장한다
- IPMI 접근을 위한 해당 계정의 패스워드를 입력받는다
- for문을 돌면서 ipmitool 명령어를 이용하여 해당 서버의 전원을 체크한다
- 모니터링 결과를 echo로 출력한다

##### 스크립트 생성


In [ ]:
%%bash
#!/bin/bash

# IPMI IP 및 User ID를 변수에 저장
ipmi_hosts="192.168.122.10 192.168.122.11 192.168.122.12 192.168.122.13"
ipmi_userid="admin"

# IPMI User ID에 해당하는 패스워드를 입력받음.
read -p "Please input ipmi password : " ipmi_pw
# 패스워드 입력을 안했으면 입력을 하지 않았다는 메시지를 보여주고, 스크립트 종료
if [[ -z $ipmi_pw ]]; then echo "You didn't input ipmi password. Please retry."; exit; fi

# ipmitool 명령어를 이용하여 해당 서버의 전원 체크
for host in $ipmi_hosts
do
  echo "#### IPMI HOST:: $host ####"
  power_stat=$(ipmitool -I lanplus -H $host -L ADMINISTRATOR -U $ipmi_userid -P $impi_pw -v power status)
  echo "$power_stat"
done


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash chk_ipmi.sh

### 11.2 네트워크 상태 모니터링할 때
##### 상황 
외부에 서비스할 경우 외부에서 유입되는 경로로 네트워크 상태를 모니터링하는 일은 매우 중요하다.  
가상화가 되면서 많은 가상 시스템들이 하나의 물리 네트워크 인터페이스를 통해 통신되므로, 네트워크에 문제가 생기면 서비스에 많은 영향을 미친다.  

##### 방법 찾기
ip address show 혹은 ifconfig로 확인할 수 있다.  
필요한 정보  
- 네트워크 모니터링 대상 서버 정보
- SSH를 통해 접근할 수 있는 전용 계정
- 네트워크 모니터링 대상 인터페이스 정보
- 네트워크 인터페이스 상태 확인 명령어: ip address show, ip link show

프로세스
- 네트워크 모니터링 대상 서버 정보를 변수에 저장한다
- 네트워크 모니터링 대상 인터페이스 정보를 변수에 저장한다
- 네트워크 인터페이스 상태 확인 명령어를 이용해 네트워크 상태를 모니터링한다
- echo 명령어로 결과를 보여준다

##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

# 네트워크 모니터링 대상서버 정보 저장
hosts="host01 host02 host03"
nic_name="eno1 eno3 enp24s0f0 enp24s0f1"

# 모니터링 대상 NIC을 검색하기 위한 grep 옵션 생성
grep_nic=""
for nic in $nic_name; do
  grep_nic=$(echo "$grep_nic -e $nic") 
done

# For문을 돌면서 네트워크 상태 체크
for host in $hosts; do
  echo "#### HOST:: $host ####"
  down_link=$(ssh mon@$host "ip link show | grep $grep_nic | grep 'state DOWN' | awk -F ': ' '{print \$2}'")
  down_link_cnt=$(ssh mon@$host "ip link show | grep $grep_nic | grep 'state DOWN' | wc -l")

  # 네트워크 상태 체크 결과 출력
  if [[ $down_link_cnt -eq 0 ]]; then
    echo "Network States are normal."
  else
    echo "Network $down_link is down. Please check network status." 
  fi
done

In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash chk_network.sh

### 11.3 서비스 상태 체크할 떄
##### 상황 
IPMI로 전원을 체크하고, 네트워크 상태를 확인했다면, 이제 시스템에 설치되어 있는 중요 서비스들의 상태를 확인해야 한다.  
서비스가 정상적으로 실행중인지 아니면 시스템의 문제로 인해 서비스가 정지되어 있는지 확인해야 한다.  

##### 방법 찾기
systemctl로 실행중인지 여부를 확인할 수 있다. netstat으로 해당 포트가 서비스중인지 확인할 수 있다.    
필요한 정보  
- 서비스 모니터링 대상 서버 정보
- SSH를 통해 접근할 수 있는 전용 계정
- 모니터링 대상 서비스들
- 모니터링 대상 서비스 포트들
- 서비스 상태 체크 명령어: systemctl
- 서비스 포트 체크 명령어: netstat

프로세스
- 서비스 모니터링 대상 서버 정보들을 변수에 저장한다
- 모니터링 대상 서비스들을 변수에 저장한다
- 모니터링 대상 포트들을 변수에 저장한다
- for문을 돌면서 대상 서버의 서비스 상태와 포트를 체크한다
- echo 명령어를 이용하여 모니터링 결과를 보여준다
##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

# 서비스 모니터링 대상서버 정보 저장
hosts="host01 host02 host03"
services="httpd haproxy rabbitmq"
ports="80 443 8080 5672 15672"

for host in $hosts; do
  echo "#### HOST:: $host ####"
  # 호스트별 서비스 상태 체크
  for service in $services; do
    chk_service=$(ssh mon@$host sudo systemctl is-active $service)
    # 서비스 상태 체크 결과 출력
    if [[ $chk_service == "active" ]]; then
      echo "$service state is active."
    else
      echo "$service state is inactive. Please check $service"
    fi
  done

  # 호스트별 서비스 포트 상태 체크
  for port in $ports; do
    chk_port=$(ssh mon@$host sudo netstat -ntpl | grep $port | wc -l)
    if [[ $chk_port > 0 ]]; then
       echo "This port $port is open."
    else
       echo "This port $port is not found. Please check your system."
    fi
  done
done

In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash chk_service.sh

### 11.4 클러스터 관리 툴 Pacemaker 상태 체크할 때
##### 상황 
리눅스에서 HA를 구성할 때 pacemaker라는 클러스터 관리 툴을 주로 이용한다. 시스템을 이중화할 경우 pacemaker에 해당 시스템을 자원으로 등록하면 pacemaker가 해당 시스템에 문제가 발생했을 경우 시스템을 재시작해 주기도 하고, 주요 시스템에 문제가 발생했을 경우 시스템을 재부팅시켜 주기도 한다. 따라서 대상 시스템이 pacemaker를 사용하고 있다면 해당 pacemaker의 상태도 함께 모니터링해야 한다.   

##### 방법 찾기
여러 클러스터의 pacemaker 상태를 확인할 경우 셸 스크립트로 쉽게 모니터링한다
필요한 정보   
- Pacemaker가 구성되어 있는 서버 주소
- SSH를 통해 접근할 수 있는 전용 계정
- Pacemaker 상태 체크 명령어: pcs status

프로세스
- 모니터링할 대상 서버 주소를 변수에 저장한다
- for문을 돌면서 대상 서버의 Pacemaker 상태 체크를 한다
- echo 명령어를 이용해 해당 결과를 보여준다
##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

# Pacemaker 모니터링 대상서버 정보 저장
hosts="cluster01 cluster02"

for host in $hosts; do
  echo "#### HOST:: $host ####"
  # pacemaker 상태 체크
  chk_cluster=$(ssh -q mon@$host sudo pcs status | grep 'failed' | wc -l)
  
  # pacemaker 상태 체크 결과가 없으면 문제가 없는것으로 인식
  if [[ $chk_cluster -eq 0 ]]
  then
     echo "Pacemaker status is normal."
  # pacemaker 상태 체크 결과가 있으면 문제가 있으므로 pacemaker 상태를 보여줌.
  else
     echo "Please check pacemaker status."
     echo "***********************************"
     echo "$(ssh -q mon@$host sudo pcs status)"
  fi
done


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash chk_pacemaker.sh

### 11.5 CPU 및 메모리 상태 체크할 때
##### 상황 
CPU, 메모리는 늘 확인이 필요하다. 자원 증설이나 자원 축소등의 계획을 세울 수 있다.  

##### 방법 찾기
top으로 실시간 모니터링할 수 있지만, 셸 스크립트는 mpstat, free로 체크한다
필요한 정보   
- 모니터링할 대상 서버 정보
- SSH를 통해 접근할 수 있는 전용 계정
- CPU 상태 체크 명령어: mpstat
- 메모리 상태 체크 명령어: free

프로세스
- 모니터링할 대상 서버 주소를 변수에 저장한다
- for문을 돌면서 CPU 사용률과 메모리 사용률을 체크한다
- 모니터링 결과를 echo 명령어로 출력한다
##### 스크립트 생성
mpstat 명령어는 기본 제공 명령어가 아니므로 사용하기 위해서는 sysstat을 설치해야 한다

In [ ]:
#!/bin/bash

# 모니터링 대상서버 정보 저장
hosts="host01 host02 host03"

for host in $hosts; do
  echo "#### HOST:: $host ####"
  # cpu, memory 사용률 체크
  chk_cpu=$(ssh -q mon@$host mpstat | grep all | awk '{print $4}')
  chk_mem=$(ssh -q mon@$host free -h | grep Mem | awk '{print $4}')
  
  # cpu, memory 사용률 체크 결과
  echo "CPU usage is ${chk_cpu}%"
  echo "Memory free size is ${chk_mem}"
done


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash chk_cpumem.sh

### 11.6 서비스 로그 모니터링할 때
##### 상황 
서비스의 실행 상태 말고, 코드에 의한 버그나 잘못 입력한 파라미터, 관련 시스템의 문제로 인한 에러 로그 등을 모니터링해야 한다.  
수많은 서비스가 있다면 서비스들의 로그를 전부 모니터링하는 것은 쉬운 일이 아니다.  
전용 모니터링 시스템만큼은 아니지만 그럭저럭 셸 스크립트로 할 수 있다  

##### 방법 찾기
서비스 로그 모니터링을 하기 위해서는 서비스 로그가 쌓이는 대상 디렉터리를 알고 있어야 한다.
필요한 정보   
- 서비스 로그 모니터링 대상 서버 정보
- SSH를 통해 접근할 수 있는 전용 계정
- 서버별 실행 서비스 및 로그 디렉터리 정보

프로세스
- 서비스 로그 모니터링 대상 서버 정보들을 변수에 저장한다
- for문을 돌면서 대상 서버의 서비스 로그를 확인한다
- 확인된 결과를 echo로 출력한다

##### 스크립트 생성
모든 로그를 모니터링하기보다는 ERROR 로그를 검색하여 모니터링하는 것이 좋다

In [ ]:
#!/bin/bash

# 모니터링 대상서버 정보 저장
hosts="host01 host02 host03"
# 모니터링 대상 서비스 정보 저장
services="httpd rabbitmq nginx"

for host in $hosts; do
  echo "#### HOST:: $host ####"

  for service in $services; do
    # service's error log 검색
    chk_log=$(ssh mon@$host sudo tail /var/log/$service/*.log | grep -i error | wc -l)
    
    # error log가 없으면 없다고 메시지를 보여줌.
    if [[ $chk_log -eq 0 ]]; then
      echo "No error services logs. The $service is normal"
    else
      # error log가 있는 경우에는 체크하라고 메시지를 보여줌.
      echo "Please check service $service logs and service $service"
      echo "$(ssh mon@$host sudo tail /var/log/$service/*.log | grep -i error)"
    fi
  done
done


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash chk_servicelog.sh

### 11.7 컨테이너 실행 상태 모니터링할 때
##### 상황 
시스템에서 컨테이너를 이용해 서비스를 한다면, 컨테이너의 실행 상태 역시 모니터링이 필요하다.  
Docker 서비스 상태도 확인이 필요하다  

##### 방법 찾기
컨테이너 엔진이 Docker인지 Podman인지 알아야 한다
필요한 정보   
- 컨테이너 서비스가 운영 중인 서버 정보
- 컨테이너 엔진 확인 명령어: systemctl list-units
- 컨테이너 실행 상태 확인 명령어: docker ps 혹은 podman ps

프로세스
- 컨테이너 서비스가 운영중인 서버 정보를 변수에 저장한다
- for문을 돌면서 어떤 컨테이너 엔진이 실행중인지 확인한다
- 컨테이너 실행 상태를 확인한다
- 모니터링 결과를 echo를 이용해 보여준다

##### 스크립트 생성

In [ ]:
#!/bin/bash

# 모니터링 대상서버 정보 저장
hosts="host01 host02 host03"

for host in $hosts;do
  echo "#### HOST:: $host ####"
  # 해당 호스트에 Docker가 설치되어 있는지 확인
  chk_docker=$(ssh -q mon@$host rpm -qa | grep -c docker)
  if [[ $chk_docker > 0]]; then
    echo "This system's container engine is docker."
    # docker 서비스가 실행중인지 확인
    chk_service=$(ssh -q mon@$host systemctl is-active docker)
    if [[ $chk_service == "active" ]]; then
      echo "docker running state is active."
    else
      echo "Please check your docker state."
    fi
    # container 프로세스 확인
    chk_container=$(ssh -q mon@$host docker ps | grep -c seconds)  #초가 계속 반복되는 컨테이너는 문제가 있는 것으로 판정 (restart 걸어놓음)
    if [[ $chk_container > 0 ]]; then
      echo "Please check your container state."
      echo "$(ssh -q mon@$host docker ps | grep seconds)"
    else
      echo "Container status is normal."
    fi
  fi

 # 해당 호스트에 Docker가 설치되어 있는지 확인
  chk_podman=$(ssh -q mon@$host rpm -qa | grep -c podman)
  if [[ $chk_podman > 0]]; then
    echo "This system's container engine is podman."
    # container 프로세스 확인
    chk_container=$(ssh -q mon@$host podman ps | grep -c seconds)
    if [[ $chk_container > 0 ]]; then
      echo "Please check your container state."
      echo "$(ssh -q mon@$host podman ps | grep seconds)"
    else
      echo "Container status is normal."
    fi
  fi
done


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash chk_container.sh

### \[종합\]11.8 일일 시스템 점검 모니터링하기
##### 상황 


##### 방법 찾기
ㅜㅜ
필요한 정보   
- 

프로세스
- 

##### 스크립트 생성

#!/bin/bash

#-----------------------------
# 모니터링 대상 서버정보를 변수에 저장
#-----------------------------
Cluster_servers="clus01 clus02 clus03"
Container_servers="con01 con02 con03"
General_servers="gen01 gen02 gen03 gen04 gen05"

LOG_FILE=""
#-----------------------------
# 모니터링 로그 파일 생성
#-----------------------------
function make_logs()
{
  DATE=$(date +%Y%m%d%H%M)
  LOG_FILE="/var/log/daily_system_chk/chk_system_$DATE.log"
  sudo touch $LOG_FILE
  sudo chmod 777 $LOG_FILE
}

#-----------------------------
# 모니터링 로그 파일 권한 변경
#-----------------------------
function change_logs()
{
  sudo chmod 644 $LOG_FILE
}

#-----------------------------------
# 모니터링 결과를 화면에 출력하고 로그에 저장
#-----------------------------------
function print_msg()
{
  Message=$1
  Date=$(date "+%Y-%m-%d %H:%M")
  echo "$Date [Daily_System_chk] $Message" >> $LOG_FILE
  echo "$Date $Message" 
}

make_logs
print_msg "#-----------------------------"
print_msg "# Check System Power"
print_msg "#-----------------------------"

for i in {1..3}
do
  print_msg "#### NODE:: clus0$i ####"
  power_stat=$(ipmitool -I lanplus -H 192.168.0.1$i -L ADMINISTRATOR -U admin -P P@ssw0rd! -v power status)
  print_msg "$power_stat"
done

for i in {1..3}
do
  print_msg "#### NODE:: con0$i ####"
  power_stat=$(ipmitool -I lanplus -H 192.168.0.2$i -L ADMINISTRATOR -U admin -P P@ssw0rd! -v power status)
  print_msg "$power_stat"
done

for i in {1..5}
do
  print_msg "#### NODE:: gen0$i ####"
  power_stat=$(ipmitool -I lanplus -H 192.168.0.3$i -L ADMINISTRATOR -U admin -P P@ssw0rd! -v power status)
  print_msg "$power_stat"
done

print_msg "#-----------------------------"
print_msg "# Cluster Servers"
print_msg "#-----------------------------"

for i in $Cluster_servers
do
  print_msg "#### NODE:: $i ####"

  if [ $i = "clus01" ]
  then
    print_msg "#-----------------------------"
    print_msg "# Check Clustering"
    print_msg "#-----------------------------"
    cluster_stat=$(ssh -q mon@$i sudo pcs status | grep 'failed' | wc -l)

    if [ $cluster_stat -eq 0 ]
    then
      print_msg "Pacemaker status is normal"
    else
      print_msg "Please check pacemaker"
      print_msg "$(ssh -q mon@$i sudo pcs status)"
    fi
  fi

  print_msg "#-----------------------------"
  print_msg "# Check Network"
  print_msg "#-----------------------------"
  grep_nic="-e eno1 -e eno3 -e enp24s0f0 -e enp24s0f1"
  down_link=$(ssh mon@$host "ip link show | grep $grep_nic | grep 'state DOWN' | awk -F ': ' '{print \$2}'")
  down_link_cnt=$(ssh mon@$host "ip link show | grep $grep_nic | grep 'state DOWN' | wc -l")
  if [[ $down_link_cnt -eq 0 ]]; then
    print_msg "Network States are normal."
  else
    print_msg "Network $down_link is down. Please check network status." 
  fi

  print_msg "#-----------------------------"
  print_msg "# Check CPU"
  print_msg "#-----------------------------"
  mem_stat=$(ssh -q mon@$i sudo mpstat | grep all | awk '{print $4}')
  print_msg "CPU usage is ${chk_cpu}%. If CPU usage is high, please check system cpu status."  

  print_msg "#-----------------------------"
  print_msg "# Check Memory"
  print_msg "#-----------------------------"
  mem_stat=$(ssh -q mon@$i sudo free -h | grep -i mem | awk '{print $4}')
  print_msg "Memory free size is $mem_stat. If memory free size is low, please check system memory status."  

  print_msg "#-----------------------------"
  print_msg "# Check Service Log"
  print_msg "#-----------------------------"
  for service in "httpd mariadb"; do
    chk_log=$(ssh mon@$host sudo tail /var/log/$service/*.log | grep -i error | wc -l)
    
    if [[ $chk_log -eq 0 ]]; then
      echo "No error services logs. The $service is normal"
    else
      echo "Please check service $service logs and service $service"  <------ 02
      echo "$(ssh mon@$host sudo tail /var/log/$service/*.log | grep -i error)"
    fi
  done
done

print_msg "#-----------------------------"
print_msg "# Container Servers"
print_msg "#-----------------------------"

for i in $Container_servers
do
  print_msg "#### NODE:: $i ####"

  print_msg "#-----------------------------"
  print_msg "# Check Network"
  print_msg "#-----------------------------"
  grep_nic="-e eno1 -e eno3 -e enp24s0f0 -e enp24s0f1"
  down_link=$(ssh mon@$host "ip link show | grep $grep_nic | grep 'state DOWN' | awk -F ': ' '{print \$2}'")
  down_link_cnt=$(ssh mon@$host "ip link show | grep $grep_nic | grep 'state DOWN' | wc -l")
  if [[ $down_link_cnt -eq 0 ]]; then
    print_msg "Network States are normal."
  else
    print_msg "Network $down_link is down. Please check network status." 
  fi

  print_msg "#-----------------------------"
  print_msg "# Check CPU"
  print_msg "#-----------------------------"
  mem_stat=$(ssh -q mon@$i sudo mpstat | grep all | awk '{print $4}')
  print_msg "CPU usage is ${chk_cpu}%. If CPU usage is high, please check system cpu status"  

  print_msg "#-----------------------------"
  print_msg "# Check Memory"
  print_msg "#-----------------------------"
  mem_stat=$(ssh -q mon@$i sudo free -h | grep -i mem | awk '{print $4}')
  print_msg "Memory free size is $mem_stat. If memory free size is low, please check system memory status"  

  chk_docker=$(ssh -q mon@$host rpm -qa | grep -c docker)
  if [[ $chk_docker > 0]]; then
    print_msg "#-----------------------------"
    print_msg "# Check Container - Docker"
    print_msg "#-----------------------------"
    chk_service=$(ssh -q mon@$host systemctl is-active docker)
    if [[ $chk_service == "active" ]]; then
      print_msg "Docker running state is active."
      chk_container=$(ssh -q mon@$host docker ps | grep -c seconds)
      if [[ $chk_container > 0 ]]; then
        print_msg "Please check your container state."
        print_msg "$(ssh -q mon@$host docker ps | grep seconds)"
      else
        print_msg "Container status is normal."
      fi
    else
      print_msg "Please check your docker state."
    fi
  fi

  chk_podman=$(ssh -q mon@$host rpm -qa | grep -c podman)
  if [[ $chk_podman > 0]]; then
    print_msg "#-----------------------------"
    print_msg "# Check Container - Podman"
    print_msg "#-----------------------------"
    chk_container=$(ssh -q mon@$host podman ps | grep -c seconds)
    if [[ $chk_container > 0 ]]; then
      print_msg "Please check your container state."
      print_msg "$(ssh -q mon@$host podman ps | grep seconds)"
    else
      print_msg "Container status is normal."
    fi
  fi
done

print_msg "#-----------------------------"
print_msg "# General Servers"
print_msg "#-----------------------------"

for i in $General_servers
do
  print_msg "#### NODE:: $i ####"

  print_msg "#-----------------------------"
  print_msg "# Check Network"
  print_msg "#-----------------------------"
  grep_nic="-e eno1 -e eno3 -e enp24s0f0 -e enp24s0f1"
  down_link=$(ssh mon@$host "ip link show | grep $grep_nic | grep 'state DOWN' | awk -F ': ' '{print \$2}'")
  down_link_cnt=$(ssh mon@$host "ip link show | grep $grep_nic | grep 'state DOWN' | wc -l")
  if [[ $down_link_cnt -eq 0 ]]; then
    print_msg "Network States are normal."
  else
    print_msg "Network $down_link is down. Please check network status." 
  fi

  print_msg "#-----------------------------"
  print_msg "# Check CPU"
  print_msg "#-----------------------------"
  mem_stat=$(ssh -q mon@$i sudo mpstat | grep all | awk '{print $4}')
  print_msg "CPU usage is ${chk_cpu}%. If CPU usage is high, please check system cpu status"  

  print_msg "#-----------------------------"
  print_msg "# Check Memory"
  print_msg "#-----------------------------"
  mem_stat=$(ssh -q mon@$i sudo free -h | grep -i mem | awk '{print $4}')
  print_msg "Memory free size is $mem_stat. If memory free size is low, please check system memory status"  

done
change_logs


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash daily-system-chk.sh